# Voronoi tessellation

### Turn coordinates into arrays

In [ ]:
coords = np.array(
    list(
        [list(xy) for xy in zip(ant_points.lon, ant_points.lat)]))

### Clipping Voronoi diagram to shape of Senegal

In [ ]:
poly_shapes, pts, poly_to_pt_assignments = voronoi_regions_from_coords(coords, map_commune.unary_union)

In [ ]:
map_voronoi = gpd.GeoDataFrame(poly_to_pt_assignments, columns = ['voronoi_id'], crs = map_commune.crs, geometry = poly_shapes)

### Calculating voronoi area sizes

In [ ]:
map_voronoi['voronoi_km2'] = map_voronoi.to_crs({'init': 'epsg:31028'}).area / 10**6

In [ ]:
map_voronoi_points = gpd.tools.sjoin(map_voronoi, map_ant_points, how="left", op='intersects').drop(columns = ['index_right'])

### Intersecting with commune map

In [ ]:
map_voronoi_points['geometry'] = map_voronoi_points.geometry.convex_hull

In [ ]:
map_intersection_voronoi = gpd.overlay(map_voronoi_points, map_commune[['MAP_ID', 'geometry', 'SUPERFICIE']], how = 'intersection')

In [ ]:
map_intersection_voronoi.insert(loc=0, column='intersection_id', value=list(range(len(map_intersection_voronoi))))

### Calculating relevant area sizes

In [ ]:
map_intersection_voronoi['intersection_km2'] = map_intersection_voronoi.to_crs({'init': 'epsg:31028'}).area / 10**6

In [ ]:
map_intersection_voronoi.head(3)

### Calculating spatial weight

In [ ]:
map_intersection_voronoi['w_geo_adm'] = map_intersection_voronoi.intersection_km2 / map_intersection_voronoi.SUPERFICIE
map_intersection_voronoi['w_geo_ant'] = map_intersection_voronoi.intersection_km2 / map_intersection_voronoi.voronoi_km2

### Calculating GUF weights

In [ ]:
results = []

if __name__ == "__main__":
    start_time = time.time()  
    
    pool = mp.Pool(processes=mp.cpu_count()-1)
    [pool.apply_async(extract_pixel_counts, args=(
        file_guf, map_intersection_voronoi, map_commune, map_voronoi_points, x), callback=collect_results) for x in range(0, len(map_intersection_voronoi), 1000)]
    pool.close()
    pool.join()
    
    map_intersection_voronoi = map_intersection_voronoi.merge(
        pd.DataFrame(data = results, columns = ['intersection_id', 'w_guf_adm', 'w_guf_ant']), on = 'intersection_id', how = 'left')
    print("--- %s seconds ---" % (time.time() - start_time))

### Calculating WPG weights

In [ ]:
results = []

if __name__ == "__main__":
    start_time = time.time()  
    
    pool = mp.Pool(processes=mp.cpu_count()-1)
    [pool.apply_async(extract_grid_counts, args=(
        file_wpg, map_intersection_voronoi, map_commune, map_voronoi_points, x), callback=collect_results) for x in range(0, len(map_intersection_voronoi), 1000)]
    pool.close()
    pool.join()
    
    map_intersection_voronoi = map_intersection_voronoi.merge(
        pd.DataFrame(data = results, columns = ['intersection_id', 'w_wpg_adm', 'w_wpg_ant']), on = 'intersection_id', how = 'left')
    print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
map_intersection_voronoi.to_csv('./midsave/map_voronoi.csv', index = False)
map_intersection_voronoi.to_file('./midsave/map_voronoi.shp')
map_voronoi_points.to_file('./midsave/map_voronoi_points.shp')